In [ ]:
from pathlib import Path
import mne
import sys
parentPath = Path.cwd().parent
sys.path.insert(0, str(parentPath)) 

In [ ]:
import sys, importlib.util, os
print("Root on path?", any(p.endswith("Motor-Imagery-EEG-Classifier") for p in sys.path))
print("Found src?", importlib.util.find_spec("src") is not None)
print("PYTHONPATH=", os.environ.get("PYTHONPATH"))


In [ ]:
from src.utils_io import set_seeds
set_seeds(42)

In [ ]:
rawData = Path(f"{parentPath}/data/raw/MNE-eegbci-data/files/eegmmidb/1.0.0/S001/S001R03.edf")
raw = mne.io.read_raw_edf(str(rawData), preload=True, verbose=False)
print(raw)
print("Sampling rate:", raw.info["sfreq"], "Hz")

In [ ]:
raw.info

In [ ]:
# basic clean: notch + bandpass + average reference
raw_proc = raw.load_data()
# power noise =  60 Hz for US, 50 elsewhere
raw_proc.notch_filter([60], verbose=False)
raw_proc.filter(l_freq=1., h_freq=40., phase='zero', fir_design='firwin', verbose=False)
raw_proc, _ = mne.set_eeg_reference(raw_proc, 'average')
print(raw_proc)


In [ ]:
# visualize raw data
mne.viz.plot_raw(raw_proc, duration=5.0, n_channels=16, scalings="auto", title="Cleaned EEG (5s)")


In [ ]:
raw.plot_psd(fmin=1, fmax=40, average=True)

In [ ]:
# quick PSD (Welch)

raw_proc.plot_psd(fmin=1, fmax=40, average=True)


: 